In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import backend as K
#### Flask Deployment
import time
from flask import Flask,request,jsonify
from  flask_restful import Api,Resource
import pickle
import os
import nltk
# nltk.download('stopwords')
from nltk.corpus import stopwords
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
##### Preprocess Functions #####
stop_words = stopwords.words('english')
imp_stopwords=['not','against','until']
for word in imp_stopwords:
    stop_words.remove(word)
# print(stop_words)

def text_process(text):
    ''' Pre process and convert texts to a list of words '''
    text = str(text)
    text = text.lower()

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()
    text = [word for word in text if word not in stop_words]
    text = ' '.join(text)
    return text
#### For training
def mandist(left, right):
    ''' Helper function for the similarity estimate of the LSTMs outputs'''
    return K.exp(-K.sum(K.abs(left-right), axis=1, keepdims=True))


In [40]:
tokenizer = pickle.load(open('BILSTM_stop_tokenizer','rb'))
model = tf.keras.models.load_model('BILSTM_stop.h5')
MAX_LEN=109

In [24]:
with open('nva_phrases.txt','r') as file:
    data = file.read()
data= data.split("\n")
candidates = pd.DataFrame(data[:-1],columns=['question1'])
candidates['processed'] = candidates['question1'].apply(text_process)
candidates_seq = tokenizer.texts_to_sequences(candidates.processed)
candidates_pad = pad_sequences(candidates_seq,maxlen=MAX_LEN)

In [58]:
def give_prediction(question):
    question_p = text_process(question)
    question_seq = tokenizer.texts_to_sequences([question_p])
    question_pad = pad_sequences(question_seq,maxlen=MAX_LEN)
    question_pad_n =np.array(question_pad.tolist()*len(candidates_pad))
    pred = np.argmax(model.predict([question_pad_n,candidates_pad]))
    result = candidates.iloc[pred].question1
    return result

In [59]:
test = "The parathas were amazing"
give_prediction(test)

'sizzlers are really great'